In [3]:
import numpy as np
import torch
import torch.nn as nn
from konlpy.tag import Okt
import gensim
import os

# ---------------------------------------------------------
# 1. 형태소 분석 및 단어 사전(Vocabulary) 구축
# ---------------------------------------------------------
jvm_path = r'C:\Program Files\Java\jdk-25.0.2\bin\server\jvm.dll'
os.environ['JAVA_HOME'] = r'C:\Program Files\Java\jdk-25.0.2'   # 꼭 추가

okt = Okt()
reviews = [
    "웨이팅이 너무 길었지만 존맛탱이네요! 가성비 최고.",
    "직원들이 불친절하고 맛도 그냥 그랬어요. 재방문 의사 없음"
]

# 토큰화 진행
tokenized_reviews = [okt.morphs(review, stem=True) for review in reviews]

# 데이터셋에 등장한 모든 단어를 모아 고유 인덱스 부여
word_to_idx = {"<PAD>": 0, "<UNK>": 1}
for tokens in tokenized_reviews:
    for word in tokens:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)

vocab_size = len(word_to_idx)
print(f"구축된 단어 사전 크기: {vocab_size}")

# ---------------------------------------------------------
# 2. 사전 학습된 Word2Vec 로드 및 Embedding Matrix 생성
# ---------------------------------------------------------
# 주의: 사전에 학습된 ko.bin 또는 ko.model 파일이 다운로드 되어 있어야 합니다.
# (예: Kyubyong의 한국어 Word2Vec 모델)
print("Word2Vec 모델 로딩 중...")
try:
    # gensim을 이용해 가벼운 모델 로드
    word2vec_model = gensim.models.Word2Vec.load('') 
    embed_dim = word2vec_model.vector_size
except FileNotFoundError:
    print("모델 파일이 없어 임시 차원(100)으로 진행합니다.")
    word2vec_model = {} # 실제 환경에서는 다운로드한 모델이 들어갑니다.
    embed_dim = 100

# 모델의 가중치를 담을 빈 행렬 생성 (vocab_size x embed_dim)
embedding_matrix = np.zeros((vocab_size, embed_dim))

# 다이닝코드 단어 사전을 순회하며 Word2Vec에서 벡터값을 가져옴
oov_count = 0
for word, i in word_to_idx.items():
    if word in word2vec_model:
        # 사전 학습된 벡터가 있으면 가져옴
        embedding_matrix[i] = word2vec_model[word]
    else:
        # 사전에 없는 단어(OOV)는 정규분포를 따르는 무작위 벡터로 초기화
        # <PAD>는 0벡터로 유지
        if word != "<PAD>":
            embedding_matrix[i] = np.random.normal(scale=0.6, size=(embed_dim,))
            oov_count += 1

print(f"임베딩 매트릭스 생성 완료. OOV(사전에 없는 단어) 개수: {oov_count}")
embedding_matrix_tensor = torch.FloatTensor(embedding_matrix)

# ---------------------------------------------------------
# 3. PyTorch DeepCoNN 모델에 주입
# ---------------------------------------------------------
class DeepCoNN(nn.Module):
    def __init__(self, embedding_matrix_tensor, num_filters=100, kernel_size=3):
        super(DeepCoNN, self).__init__()
        
        # from_pretrained를 사용하여 우리가 만든 행렬을 모델의 Embedding 층으로 만듦
        # 식당 도메인에 맞게 학습되도록 freeze=False로 설정 (가중치 업데이트 허용)
        self.embedding = nn.Embedding.from_pretrained(
            embedding_matrix_tensor, 
            freeze=False, 
            padding_idx=0
        )
        
        embed_dim = embedding_matrix_tensor.shape[1]
        
        # User Network (CNN) 예시
        self.user_cnn = nn.Sequential(
            nn.Conv1d(embed_dim, num_filters, kernel_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )

    def forward(self, x):
        # x: (Batch, Seq_len)
        embedded = self.embedding(x) # (Batch, Seq_len, Embed_dim)
        embedded = embedded.permute(0, 2, 1) # CNN 입력을 위해 차원 변경: (Batch, Embed_dim, Seq_len)
        out = self.user_cnn(embedded)
        return out.squeeze(2)

# 모델 생성 테스트
model = DeepCoNN(embedding_matrix_tensor)
print("DeepCoNN 임베딩 레이어 연동 완료!")

구축된 단어 사전 크기: 25
Word2Vec 모델 로딩 중...
모델 파일이 없어 임시 차원(100)으로 진행합니다.
임베딩 매트릭스 생성 완료. OOV(사전에 없는 단어) 개수: 24
DeepCoNN 임베딩 레이어 연동 완료!


In [2]:
from gensim.models import KeyedVectors
# 압축을 푼 ko.bin 파일의 경로를 지정합니다.
print("Kyubyong Word2Vec 로딩 중...")
ft_model = KeyedVectors.load_word2vec_format(
    '../../datafile/word2vec/cc.ko.300.vec', 
    binary=False, 
    limit=50000)
print("로딩 완료!")

# 단어 벡터 확인 예시
vector = ft_model['맛집']
print(vector.shape) # 보통 (200,) 차원을 가집니다.

Kyubyong Word2Vec 로딩 중...
로딩 완료!
(300,)
